In [1]:
import pandas as pd
import json
import botometer
import numpy as np
from twarc import ensure_flattened
from io import TextIOWrapper

/home/emanuele/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rapidapi_key = "41a068940amshbb2b1dd060f248fp1aefeajsn440210f5c285"
twitter_app_auth = {
    'consumer_key': '0MlLOUFkbla3mrGwwnCfG4w39',
    'consumer_secret': 'OnTZFVioHwINs0xbZO0SvpYZrKfD9HYtSfakEE9ZbZOyuU5v2R',
    'access_token': '921142165627133953-lL43r6t5Qrx8Ib0YBD9hM5w9yXJGmzG',
    'access_token_secret': 'slXU1A4qgzeB4hB9FEIhSnursMdCFq4wIPM9848N5Lcwm',
  }
botometer_api_url = 'https://botometer-pro.p.mashape.com'
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

In [3]:
class most_active_user(object):
    def __init__(self, name,tweets,tot_tweets):
        self.username=name
        self.profile_image=tweets.iloc[0]['author.profile_image_url']
        self.id=tweets.iloc[0]['author.id']
        self.created_at=tweets.iloc[0]['author.created_at']
        self.description=tweets.iloc[0]['author.description']
        self.tot_activity=tweets['text'].count()/float(tot_tweets)*100
        self.rt=self.rt_c(tweets)
        self.t=int(tot_tweets*self.tot_activity/100-self.rt)
        self.botscore=self.botometerscore(tweets.iloc[0]['author.id'])
    def rt_c(self,tweets):
        count=0
        for row in tweets.iterrows():
            if type(row[1]['referenced_tweets'])==list and len(row[1]['referenced_tweets'])==1:
                
                
                if not pd.isna(row[1]['referenced_tweets']):
                    if row[1]['referenced_tweets'][0]['type']=='retweeted':
                        count+=1
        return count
    def botometerscore(self,uid):
        result = bom.check_account(uid)
        return result['cap']['universal']
    def to_dict(self):
        return {
            'username': self.username,
            'profile mage':self.profile_image,
            'twitter id': self.id,
            'created_at': self.created_at,
            'description': self.description,
            'activity share(%)': round(self.tot_activity,2),
            'retweets': self.rt,
            'tweets': self.t,
            'botometer score': self.botscore
        }

In [4]:
results=[]
with open('../twitterderecha/voxcumple/voxcumple_may22.json', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    results.append(json.loads(json_str))
    #print(f"result: {result}")
    #print(isinstance(result, dict))

In [5]:
len(results)

50259

In [6]:
datadf=pd.json_normalize(results)

In [7]:
activity_ordered_users=datadf.groupby('author.username')['id'].nunique().sort_values(ascending=False).reset_index(name='count')

In [8]:
activity_ordered_users.head(10)

,author.username,count
0,Thomy200261,1507
1,lena_asereje,1495
2,VSumergida,1075
3,maga_asereje,1069
4,MOONVICUS4,983
5,Rosa29240097,928
6,LlampugaBosch,830
7,EstherBrilla,815
8,federico_G_,804
9,HernandoPaco,802


In [10]:
maus=[]
tot_tweets=datadf['id'].count()
for row1 in activity_ordered_users.head(10).iterrows():
    rslt_df = datadf[datadf['author.username'] == row1[1]['author.username']]
    maus.append(most_active_user(rslt_df.iloc[0]['author.username'],rslt_df,tot_tweets))
mausdf=pd.DataFrame.from_records([s.to_dict() for s in maus])

In [11]:
mausdf['activity share(%)'].sum()

20.520000000000003

In [12]:
mausdf.head()

,username,profile mage,twitter id,created_at,description,activity share(%),retweets,tweets,botometer score
0,Thomy200261,https://pbs.twimg.com/profile_images/148023990...,1102167817779138560,2019-03-03T11:24:19.000Z,♦️♣️♥️♠️ \nCualquier día es bueno para morir s...,3.00,1013,493,0.813043
1,lena_asereje,https://pbs.twimg.com/profile_images/150652820...,1367746080612495362,2021-03-05T07:58:47.000Z,#𝖳𝖾𝖺𝗆𝖵𝗈𝗑⚔💚⚔\n\n..ɱเร σʝσร ɳσ ƭเεɳεɳ ∂µεɳ̃σ\nρσ...,2.97,1074,421,0.825257
2,VSumergida,https://pbs.twimg.com/profile_images/148338577...,1271312380807442432,2020-06-12T05:26:54.000Z,,2.14,1026,48,0.832786
3,maga_asereje,https://pbs.twimg.com/profile_images/148810218...,1367187663606992897,2021-03-03T18:59:22.000Z,#𝖳𝖾𝖺𝗆𝖵𝗈𝗑⚔💚⚔\n\ndicen.... que los vivos pueden ...,2.13,888,181,0.805343
4,MOONVICUS4,https://pbs.twimg.com/profile_images/148172181...,887282370033983488,2017-07-18T12:06:16.000Z,Rebelde. Conservador/Liberal y/o viceversa.\n¡...,1.96,44,939,0.808228


In [14]:
mausdf.index = np.arange(1, len(mausdf)+1)
mausdf.to_csv('../twitterderecha/voxcumple/voxcumple_may22_maus.csv')

In [10]:
urls=[]
for u in datadf['entities.urls']:
    if type(u)==list:
        for uu in u:
            if 'twitter.com' not in uu['expanded_url'].lower():
                urls.append(uu['expanded_url'].lower().replace('www.',''))
        

In [11]:
from collections import Counter

In [24]:
len(urls)

NameError: name 'SPVD8_20' is not defined

In [13]:
Counter(urls).most_common(15)

[('https://abc.es/internacional/abci-13-heroes-ucranianos-isla-serpientes-antes-morir-buque-ruso-202202251008_noticia.html#vca=rrss-inducido&vmc=abc-es&vso=tw&vli=noticia.video',
  149),
 ('https://abc.es/internacional/abci-13-heroes-ucranianos-isla-serpientes-antes-morir-buque-ruso-202202251008_noticia.html',
  136),
 ('https://cnn.it/3jbxymi', 92),
 ('https://actualidad.rt.com/actualidad/421860-ucrania-militares-isla-defensa',
  72),
 ('https://libertaddigital.com/internacional/europa/2022-02-25/los-13-heroes-de-la-isla-de-las-serpientes-antes-de-ser-bombardeados-buque-ruso-que-os-den-6870018/',
  46),
 ('http://ver.abc.es/zofuj4', 39),
 ('https://israelnoticias.com/internacional/destruido-el-buque-de-guerra-ruso-que-ataco-la-isla-de-la-serpiente/',
  29),
 ('https://bit.ly/3tfxgc2', 29),
 ('https://bit.ly/3in9je7', 29),
 ('https://abc.es/internacional/abci-13-heroes-ucranianos-isla-serpientes-antes-morir-buque-ruso-202202251008_noticia.html#vca=amp-rrss-inducido&vmc=abc-es&vso=tw&vl

In [104]:

rslt_df = datadf[datadf['author.username'] == 'Nanisimo']

In [32]:

rslt_df = datadf[datadf['author.id'] == '1187822157453283329']

In [108]:
for row in rslt_df.iterrows():
    #if 'RT' not in row[1]['text']:
        print('w ',row[1]['text'],row[1]['created_at'])

w  @borja_rodriguez Jamás dije que el fantasma de Kiev fuera real. Desde el PRIMER día dije que no se podía confirmar. Sobre la Isla de la Serpiente, ya lo corregí. ¿Algo más o vais a seguir con la misma pantomima hasta el final de vuestros días?
https://t.co/dWkJCrWxnO 2022-03-12T20:58:09.000Z
w  @cleosagas1 No es lo mismo. Del fantasma de Kiev solo había un vídeo random de un caza. Aquí hay decenas de soldados con uniformes rusos y uno que graba haciendo referencia a hechos de la última semana. 2022-03-03T13:45:32.000Z
w  🇷🇺💥🇺🇦 Leo referencias de este MiG-29 ucraniano que podría haberse anotado varias victorias aire-aire en los cielos de Kiev. Lo llaman 'El fantasma de Kiev'.

No he podido comprobar por ningún sitio que sea cierto, pero es TT. ¿Quizá propaganda ucraniana? Suena muy Ace Combat. https://t.co/EV97Fn7hd1 https://t.co/VJtS1E92UC 2022-02-25T00:24:02.000Z


In [110]:
row[1]['author.description']

'Periodista maricoper🐸\nEn @Newtral.\nPolítica, cultura y tech desde Estados Unidos.\n📲 Instagram/Twitch: Nanisimo\n📩 yo@emiliodomenech.com\n📬 Mi newsletter👇🏼'

In [74]:
rslt_df

,source,conversation_id,text,lang,reply_settings,created_at,possibly_sensitive,author_id,id,entities.annotations,...,attachments.poll.voting_status,attachments.poll.id,attachments.poll.end_datetime,attachments.poll.options,attachments.poll.duration_minutes,withheld.copyright,withheld.country_codes,author.withheld.country_codes,geo.coordinates.type,geo.coordinates.coordinates
297,Twitter Web App,1506292248283123712,"RT @GeorgeArtwell: What is it? Its a bird, it'...",en,everyone,2022-03-22T15:30:32.000Z,False,2504828107,1506292248283123712,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1878,Twitter for iPhone,1503849180426555394,RT @GeorgeArtwell: “El Fantasma de Kiev” (The ...,en,everyone,2022-03-15T21:42:39.000Z,False,2504828107,1503849180426555394,"[{'start': 20, 'end': 38, 'probability': 0.839...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1879,Twitter for iPhone,1503849100827111430,RT @GeorgeArtwell: “El Fantasma de Kyiv” UNIQU...,en,everyone,2022-03-15T21:42:20.000Z,False,2504828107,1503849100827111430,"[{'start': 20, 'end': 38, 'probability': 0.358...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2626,Twitter Web App,1503413769283723270,"RT @GeorgeArtwell: #14Mar Kharkiv, El Fantasma...",es,everyone,2022-03-14T16:52:29.000Z,False,2504828107,1503413769283723270,"[{'start': 35, 'end': 53, 'probability': 0.326...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2752,Twitter for iPhone,1503161010487758852,"#14Mar Kharkiv, El Fantasma de Kyiv lo volvió ...",es,everyone,2022-03-14T00:08:06.000Z,False,2504828107,1503161010487758852,"[{'start': 16, 'end': 34, 'probability': 0.363...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3049,Twitter for iPhone,1502751439864664069,“El Fantasma de Kiev” (The Ghost of Kyiv) #NFT...,en,everyone,2022-03-12T21:00:37.000Z,False,2504828107,1502751439864664069,"[{'start': 1, 'end': 19, 'probability': 0.7781...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3418,Twitter for iPhone,1502379398070104064,"RT @GeorgeArtwell: Saludos, tengo una idea d p...",es,everyone,2022-03-11T20:22:16.000Z,False,2504828107,1502379398070104064,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3472,Twitter for iPhone,1502268022827982849,RT @GeorgeArtwell: El Fantasma de Kiev #ArtePa...,es,everyone,2022-03-11T12:59:42.000Z,False,2504828107,1502268022827982849,"[{'start': 22, 'end': 37, 'probability': 0.308...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3473,Twitter for iPhone,1502267945170485258,"RT @GeorgeArtwell: Look what I found! ""El Fant...",en,everyone,2022-03-11T12:59:23.000Z,False,2504828107,1502267945170485258,"[{'start': 68, 'end': 84, 'probability': 0.826...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3474,Twitter for iPhone,1502267929076936709,"RT @GeorgeArtwell: Look what I found! ""El Fant...",en,everyone,2022-03-11T12:59:19.000Z,False,2504828107,1502267929076936709,"[{'start': 39, 'end': 57, 'probability': 0.653...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
datadf['created_at']= pd.to_datetime(datadf['created_at'])

In [134]:
ndf=(datadf.sort_values('created_at',ignore_index=True).groupby(['author.username'], as_index=False)
     .agg(**{'First Date':('created_at', 'first'),
             'Last Date': ('created_at', 'last'),
             'Id':('author.id','first'), 
             'Description': ('author.description','first'),
             'Tot tweets':('id','count')}))

In [133]:
ndf.sort_values('First Date',ignore_index=True).head(40)

,author.username,First Date,Last Date,Id,Description,Tot tweets
0,undeadpata,2022-02-24T12:37:43.000Z,2022-02-24T12:37:43.000Z,3323313070,"esqueleto a tiempo completo, compartidor de memes, aficionado a videojuegos",1
1,metraIIeta,2022-02-24T19:49:10.000Z,2022-02-24T19:49:10.000Z,1059961568304214017,como estas carmen \n\nthey/them,1
2,DarkSalsa,2022-02-24T21:05:32.000Z,2022-02-24T21:05:32.000Z,1350796831232503810,the Cum of Kiev,1
3,sigmascoping,2022-02-24T21:06:01.000Z,2022-02-26T15:48:02.000Z,1221107425782247424,honestly just a 🤢slug🤢wrangler🤢 posting his entire stream of thought and rambling about deranged nonsense\npfp by @paguetisqueso,3
4,Danielgaay,2022-02-24T21:18:39.000Z,2022-02-26T21:30:13.000Z,1255196551753990144,"Caso procure Yuri: @bekasaaa\n→\nEu, o próprio😎!!!\n→\nSei lá",5
5,Kastro44_,2022-02-24T21:31:59.000Z,2022-03-04T13:37:37.000Z,1122342006817198080,You will live to see man-made horrors beyond your comprehension,4
6,Proschayvympel,2022-02-24T22:34:16.000Z,2022-02-24T22:34:16.000Z,1291022398800748553,amor da minha vida toda que e mto mais linda que eu @agostosaeburra\nI LOVE CRAZY FROG,1
7,falcoyeahh,2022-02-24T22:46:16.000Z,2022-02-28T02:42:29.000Z,1387925761227730949,"A negligible amount of tomfoolery, for those inclined\nミグマスター",2
8,benitofettucine,2022-02-24T23:56:30.000Z,2022-02-24T23:56:30.000Z,3234168079,"born to shit, forced to wipe",1
9,zapata131,2022-02-25T00:00:25.000Z,2022-02-25T00:00:25.000Z,15749887,"Nerd/Human lvl 34. Erdős 4.\nTRC/GDL/MUC.\nMath, Writing, Board Games.\nHablo de juegos de mesa @laMatatenaGDL.\nTech Writer at @Google.\nTweets of my own.",1


In [141]:
ndf[ndf['Description'].str.contains('periodista|Periodista')].sort_values('First Date',ignore_index=True)

,author.username,First Date,Last Date,Id,Description,Tot tweets
0,antidictadores,2022-02-25T00:17:50.000Z,2022-03-23T21:44:34.000Z,64791519,Periodista por profesión.\nPolitólogo por vocación.\nDemócrata por convicción.\nEmprendedor por adopción.,2
1,Nanisimo,2022-02-25T00:24:02.000Z,2022-03-12T20:58:09.000Z,26839592,"Periodista maricoper🐸\nEn @Newtral.\nPolítica, cultura y tech desde Estados Unidos.\n📲 Instagram/Twitch: Nanisimo\n📩 yo@emiliodomenech.com\n📬 Mi newsletter👇🏼",3
2,edu17burgos,2022-02-25T01:58:07.000Z,2022-02-25T01:58:07.000Z,289420387,⚽Periodista en @diarioAs | 📔 vicepresidente @aepducm | A veces hablo del Atleti,1
3,NY20204,2022-02-25T02:02:11.000Z,2022-02-26T12:35:58.000Z,1217490096573894657,"Periodista y analista SEO en diario https://t.co/O4DCQhLAOX.\n\nAnteriormente: La República, Mi Bundesliga (España), https://t.co/NVnXt7dMiC y La Nueve.\n\nAprendí haciendo en @isil_pe",5
4,alejo9712,2022-02-25T02:38:16.000Z,2022-02-25T02:38:16.000Z,479819012,-Comunicador social \n-Periodista\n-Fotografo\n-Rolo de pura cepa,1
...,...,...,...,...,...,...
364,Mujit_xx,2022-03-15T15:22:21.000Z,2022-03-15T15:22:21.000Z,1411430211506343939,Comunicadora social y periodista 💖,1
365,DANIELFIOCHETTA,2022-03-18T12:53:24.000Z,2022-03-18T12:53:24.000Z,1596802808,Periodista Radio Nihuil AM 680 FM 98.9,1
366,Anaopereta26,2022-03-18T17:57:49.000Z,2022-03-18T17:57:49.000Z,896527330910318592,Gobierno Política Política y Gobierno Gobierno Notícia Radio Periodistas Prensa Artistas internacionales,1
367,crisalejkauff,2022-03-19T00:25:48.000Z,2022-03-19T00:25:48.000Z,150503387,https://t.co/SwJg0tRoQB💘💘 y Peronista 🇦🇷\nDefendiendo el derecho a la informacion! Basta de sobres y periodistas trucho,1


In [127]:
ndf[ndf['author.username']=='IrinaMar10']

,author.username,First Date,Last Date,Description,Total


In [93]:
cum=pd.DataFrame(ndf.groupby([ndf["First Date"].dt.year,ndf["First Date"].dt.month,ndf["First Date"].dt.day])['author.username'].nunique().unstack(fill_value=0).stack().cumsum())

In [95]:
cum['Date']=pd.to_datetime(df1,errors='coerce')

In [97]:
eurt=pd.DataFrame(cs.div(cs.shift(1)).fillna(0))

NameError: name 'cs' is not defined

In [102]:
cum.to_csv('../../AgoraTwarc/cumuser.csv')

In [98]:
cs=ndf.groupby([ndf["First Date"].dt.year,ndf["First Date"].dt.month,ndf["First Date"].dt.day])['author.username'].nunique().unstack(fill_value=0).stack()

In [99]:
eurt=pd.DataFrame(cs.div(cs.shift(1)).fillna(0))
eurt['Date'] =  pd.to_datetime(df1,errors='coerce')

In [100]:
cum['eurt']=eurt[0]


In [54]:
cum['Date'] =  pd.to_datetime(df1,errors='coerce')

In [101]:
cum

0       Date      eurt
First Date First Date First Date                           
2021       7          1              0 2021-07-01  0.000000
                      2              0 2021-07-02  0.000000
                      3              1 2021-07-03       inf
                      4              4 2021-07-04  3.000000
                      5              5 2021-07-05  0.333333
...                                ...        ...       ...
2022       1          27          2094 2022-01-27  0.333333
                      28          2097 2022-01-28  3.000000
                      29          2100 2022-01-29  1.000000
                      30          2100 2022-01-30  0.000000
                      31          2100 2022-01-31  0.000000

[217 rows x 3 columns]

In [15]:
y = result.index.get_level_values(level=0)
m = result.index.get_level_values(level=1)
d = result.index.get_level_values(level=2)
df1 = pd.DataFrame({'year':y, 'month':m, 'day':d}, index=result.index)

NameError: name 'result' is not defined

In [85]:
y = cum.index.get_level_values(level=0)
m = cum.index.get_level_values(level=1)
d = cum.index.get_level_values(level=2)
df1 = pd.DataFrame({'year':y, 'month':m, 'day':d}, index=cum.index)

In [86]:
cum['Dates'] =  pd.to_datetime(df1,errors='coerce')

In [84]:
y = result.index.get_level_values(level=0)
m = result.index.get_level_values(level=1)
d = result.index.get_level_values(level=2)
df1 = pd.DataFrame({'year':y, 'month':m, 'day':d}, index=result.index)

NameError: name 'result' is not defined

In [58]:
for row in pd.to_datetime(df1,errors='coerce').iterrows():
    print(row)

AttributeError: 'Series' object has no attribute 'iterrows'

In [81]:
cum

First Date  First Date  First Date
2021        7           1                0
                        2                0
                        3                1
                        4                4
                        5                5
                                      ... 
2022        1           27            2094
                        28            2097
                        29            2100
                        30            2100
                        31            2100
Length: 217, dtype: int64

In [92]:
cum['Date']

First Date  First Date
                          0      2022-01-29 19:01:17+00:00
1      2022-0...
dtype: object

In [151]:
datadf

,lang,text,conversation_id,referenced_tweets,source,reply_settings,id,created_at,possibly_sensitive,author_id,...,geo.id,geo.country_code,geo.country,geo.place_type,geo.geo.type,geo.geo.bbox,in_reply_to_user.entities.description.urls,entities.annotations,geo.coordinates.type,geo.coordinates.coordinates
0,it,"@14Agosto1892 @v2Dark L'altro giorno, per la p...",1487405203649404935,"[{'type': 'replied_to', 'id': '148741722489377...",Twitter for Android,everyone,1487501119513374727,2022-01-29 19:01:17+00:00,False,1152585760551444480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,it,"Sostengo e partecipo all' @agora_dem ""Il Cinem...",1487421370426343424,NaN,Twitter Web App,everyone,1487421370426343424,2022-01-29 13:44:24+00:00,False,485951615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,it,(4/4) Per questo #Mondodem ha proposto una ser...,1487359621329297408,"[{'type': 'replied_to', 'id': '148735962434920...",Twitter for Android,everyone,1487359625850765313,2022-01-29 09:39:03+00:00,False,4793814082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,it,RT @pdabruzzo: Oggi l’@agora_dem “Se non è par...,1486972848510017537,"[{'type': 'retweeted', 'id': '1486972530728579...",Twitter for iPhone,everyone,1486972848510017537,2022-01-28 08:02:08+00:00,False,2919449735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,it,Oggi l’@agora_dem “Se non è paritaria non è #d...,1486972530728579073,NaN,Twitter for iPhone,everyone,1486972530728579073,2022-01-28 08:00:52+00:00,False,252676505,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6532,it,#Letta: “Rispondiamo a destre con agorà democr...,1411403116465909760,NaN,TweetDeck,everyone,1411403116465909760,2021-07-03 19:15:00+00:00,False,860015294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6533,it,"Enrico Letta ci riprova: ""Il lancio delle agor...",1411360951614459915,NaN,Informazione,everyone,1411360951614459915,2021-07-03 16:27:27+00:00,False,317671787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6534,it,"Pd: Letta, 'agorà democratiche per nuovo centr...",1411286892558139394,NaN,Informazione,everyone,1411286892558139394,2021-07-03 11:33:10+00:00,False,317671787,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6535,it,"Pd: Letta, 'agorà democratiche per nuovo centr...",1411283634514976769,NaN,auto-publish-website,everyone,1411283634514976769,2021-07-03 11:20:13+00:00,False,1130806238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
pd.set_option('display.max_colwidth', None)

In [146]:
with open('../fantasma_users_summary.csv','r') as f:
    uss=pd.read_csv(f)

In [147]:
uss

,author.username,First Date,Last Date,Id,Description,Tot tweets
0,undeadpata,2022-02-24T12:37:43.000Z,2022-02-24T12:37:43.000Z,3.323313e+09,"esqueleto a tiempo completo, compartidor de memes, aficionado a videojuegos",1.0
1,metraIIeta,2022-02-24T19:49:10.000Z,2022-02-24T19:49:10.000Z,1.059962e+18,como estas carmen \n\nthey/them,1.0
2,DarkSalsa,2022-02-24T21:05:32.000Z,2022-02-24T21:05:32.000Z,1.350797e+18,the Cum of Kiev,1.0
3,sigmascoping,2022-02-24T21:06:01.000Z,2022-02-26T15:48:02.000Z,1.221107e+18,honestly just a 🤢slug🤢wrangler🤢 posting his entire stream of thought and rambling about deranged nonsense\npfp by @paguetisqueso,3.0
4,Danielgaay,2022-02-24T21:18:39.000Z,2022-02-26T21:30:13.000Z,1.255197e+18,"Caso procure Yuri: @bekasaaa\n→\nEu, o próprio😎!!!\n→\nSei lá",5.0
...,...,...,...,...,...,...
44266,danielb_br,2022-03-25T00:55:32.000Z,2022-03-25T00:55:32.000Z,1.278046e+18,#NacionalGostosismo,1.0
44267,OdjoLaI66576930,2022-03-25T01:02:18.000Z,2022-03-25T01:02:18.000Z,1.370181e+18,La vie est éphémère mais belle,1.0
44268,salamanca_elkin,2022-03-25T01:28:54.000Z,2022-03-25T01:28:54.000Z,1.256661e+18,NaN,1.0
44269,lh_lucho,2022-03-25T03:38:28.000Z,2022-03-25T03:38:28.000Z,2.571452e+09,"Nací un 28-Dic, Católico, Patriota🇨🇱, Runner, ciclista, con orgullo de derecha, soy del 44%, apoyo a Carabineros de Chile, el comunismo debe ser PROSCRITO!",1.0


In [152]:
datadf.sort_values('created_at',ignore_index=True).groupby(['author.username'])

In [205]:
def my_agg92(x):
  
   names = {
       'First Date':list(x.created_at)[0],
       'Last Date': list(x.created_at)[-1],
       'Id':list(x['author.id'])[0],
       'Description': list(x['author.description'])[0],
       'Tot activity':x.id.nunique(),
       'Tot retweets': x[x.text.str.contains('RT @')]['id'].nunique(),   
      }


   return pd.Series(names, index=[k for k in names.keys()])

zres=datadf.sort_values('created_at',ignore_index=True).groupby(['author.username']).apply(my_agg92)



In [222]:
zres.iloc[['DarkSalsa']]

ValueError: invalid literal for int() with base 10: 'DarkSalsa'

In [224]:
zres[zres['author.username']=='CaraotaDigital']

KeyError: 'author.username'

In [228]:
zres.loc['CaraotaDigital']

First Date                                                           2022-02-26T22:30:35.000Z
Last Date                                                            2022-03-01T22:03:04.000Z
Id                                                                                  171299971
Description     Medio de comunicación para la comunidad de Habla Hispana. \nSomos #ADNCaraota
Tot activity                                                                              119
Tot retweets                                                                                0
Name: CaraotaDigital, dtype: object

In [235]:
with open('../fantasma_users_summary.csv','r') as f:
    zz=pd.read_csv(f)

In [236]:
zz

,author.username,First Date,Last Date,Id,Description,Tot activity
0,undeadpata,2022-02-24T12:37:43.000Z,2022-02-24T12:37:43.000Z,3.323313e+09,"esqueleto a tiempo completo, compartidor de memes, aficionado a videojuegos",1.0
1,metraIIeta,2022-02-24T19:49:10.000Z,2022-02-24T19:49:10.000Z,1.059962e+18,como estas carmen \n\nthey/them,1.0
2,DarkSalsa,2022-02-24T21:05:32.000Z,2022-02-24T21:05:32.000Z,1.350797e+18,the Cum of Kiev,1.0
3,sigmascoping,2022-02-24T21:06:01.000Z,2022-02-26T15:48:02.000Z,1.221107e+18,honestly just a 🤢slug🤢wrangler🤢 posting his entire stream of thought and rambling about deranged nonsense\npfp by @paguetisqueso,3.0
4,Danielgaay,2022-02-24T21:18:39.000Z,2022-02-26T21:30:13.000Z,1.255197e+18,"Caso procure Yuri: @bekasaaa\n→\nEu, o próprio😎!!!\n→\nSei lá",5.0
...,...,...,...,...,...,...
44266,danielb_br,2022-03-25T00:55:32.000Z,2022-03-25T00:55:32.000Z,1.278046e+18,#NacionalGostosismo,1.0
44267,OdjoLaI66576930,2022-03-25T01:02:18.000Z,2022-03-25T01:02:18.000Z,1.370181e+18,La vie est éphémère mais belle,1.0
44268,salamanca_elkin,2022-03-25T01:28:54.000Z,2022-03-25T01:28:54.000Z,1.256661e+18,NaN,1.0
44269,lh_lucho,2022-03-25T03:38:28.000Z,2022-03-25T03:38:28.000Z,2.571452e+09,"Nací un 28-Dic, Católico, Patriota🇨🇱, Runner, ciclista, con orgullo de derecha, soy del 44%, apoyo a Carabineros de Chile, el comunismo debe ser PROSCRITO!",1.0
